# Import the cleaned data (see Preprocessing.py for more info)

In [ ]:
from Preprocessing import getData
import numpy as np

x_train,x_val,x_test,y_train,y_val,y_test = getData()

print(x_train[0])

In [ ]:
print(f"train data: {len(x_train)}\t val data : {len(x_val)}\t test data: {len(x_test)}")
unique, counts = np.unique(y_train, return_counts=True)
print(f"Classes and count for x_train: {dict(zip(unique, counts))}")
print(x_train.shape)
most_probable_class = np.argmax(counts)

# Generate the model

In [ ]:
from keras import layers
from keras.models import Sequential

In [ ]:
print(x_train.shape)

model = Sequential()

model.add(layers.LSTM(units=20, return_sequences=True))
# model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units=3, dropout=0.2, activation='softmax')) # dropout, pour éviter les overfitting

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
rollingWindow = 1000
valWindow = 0
step = 500
from tqdm import tqdm
for i in tqdm(range(0,len(x_train) - rollingWindow-valWindow,step)):
    model.fit(x_train[i:i+rollingWindow], y_train[i:i+rollingWindow], epochs=1, verbose=0,validation_data=(x_train[i+rollingWindow:i+rollingWindow+valWindow], y_train[i+rollingWindow:i+rollingWindow+valWindow]))

evaluatation = model.evaluate(x_test, y_test,verbose=0)


print(f"\n\nEvaluation loss:  {evaluatation[0]} \t Evaluation accuracy: {evaluatation[1]}")

from sklearn.metrics import accuracy_score
base_line = accuracy_score(y_test, [most_probable_class]*len(y_test))
print(base_line)